In [1]:
from one.api import ONE
import numpy as np

ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

# Search for all experiments with channels.mlapdv dataset
eids = one.search(data='channels.mlapdv')
print(eids)
# Concatenate all the mlapdv values from the probe00 collections
mlapdv_concat = []
probe00_eids = []
for eid in eids:
    try:
        mlapdv = one.load_dataset(eid, 'channels.mlapdv.npy', collection='alf/probe00/pykilosort')
        mlapdv_mean = np.mean(mlapdv, axis=0) # compute the mean of the mlpadv values
        mlapdv_concat.append(mlapdv_mean)
        probe00_eids.append((eid, 'probe00'))
    except:
        pass

# Concatenate all the mlapdv values from the probe01 collections
probe01_eids = []
for eid in eids:
    try:
        mlapdv = one.load_dataset(eid, 'channels.mlapdv.npy', collection='alf/probe01/pykilosort')
        mlapdv_mean = np.mean(mlapdv, axis=0) # compute the mean of the mlpadv values
        mlapdv_concat.append(mlapdv_mean)
        probe01_eids.append((eid, 'probe01'))
    except:
        pass

# Concatenate all the mlapdv values together
mlapdv_concat = np.stack(mlapdv_concat) # stack the mlpadv values into a single array
eids = probe00_eids + probe01_eids # combine the eid and probe information into a single list

['ae8787b1-4229-4d56-b0c2-566b61a25b77', '0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde', '6c6b0d06-6039-4525-a74b-58cfaa1d3a60', '1d4a7bd6-296a-48b9-b20e-bd0ac80750a5', 'e0928e11-2b86-4387-a203-80c77fab5d52', '196a2adf-ff83-49b2-823a-33f990049c2e', '78b4fff5-c5ec-44d9-b5f9-d59493063f00', '1ca83b26-30fc-4350-a616-c38b7d00d240', '27ef44c0-acb2-4220-b776-477d0d5abd35', 'a2701b93-d8e1-47e9-a819-f1063046f3e7', '446f4724-1690-49f9-819a-2bd8e2ea88ce', 'c8d46ee6-eb68-4535-8756-7c9aa32f10e4', '8928f98a-b411-497e-aa4b-aa752434686d', '3f859b5c-e73a-4044-b49e-34bb81e96715', 'f819d499-8bf7-4da0-a431-15377a8319d5', '752456f3-9f47-4fbf-bd44-9d131c0f41aa', '1b9e349e-93f2-41cc-a4b5-b212d7ddc8df', '6b0b5d24-bcda-4053-a59c-beaa1fe03b8f', 'd901aff5-2250-467a-b4a1-0cb9729df9e2', '5b49aca6-a6f4-4075-931a-617ad64c219c', 'eacc49a9-f3a1-49f1-b87f-0972f90ee837', '3e6a97d3-3991-49e2-b346-6948cb4580fb', '1425bd6f-c625-4f6a-b237-dc5bcfc42c87', '63c70ae8-4dfb-418b-b21b-f0b1e5fba6c9', '9b528ad0-4599-4a55-9148-96cc1d93fb24',

local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy
/home/maria/Downloads/ONE/openalyx.internationalbrainlab.org/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy: 100%|██████████| 4.74k/4.74k [00:00<00:00, 11.6kB/s]


In [ ]:
np.save('data/mlapdv_concat.npy',mlapdv_concat)
np.save('data/mlapdv_eids.npy', eids)

In [ ]:
mlapdv_concat.shape

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=20).fit_transform(mlapdv_concat)
plt.scatter(X_embedded[:,0],X_embedded[:,1])

In [ ]:
np.save('/data/probe_location_embeddings.npy',X_embedded)

In [ ]:
import matplotlib.pyplot as plt
from ibllib.atlas import atlas as at
def generate_mlapdv_plots(eids):
    atlas=at.AllenAtlas()
    for i, eid in enumerate(eids):
        # Load mlapdv.npy for probe00 if it exists
        if eid[1]=='probe00':
            mlapdv = one.load_dataset(eid[0], 'channels.mlapdv.npy', collection=f'alf/probe00/pykilosort')

        # Load mlapdv.npy for probe01 if it exists
        if eid[1]=='probe01':
            mlapdv = one.load_dataset(eid[0], 'channels.mlapdv.npy', collection=f'alf/probe01/pykilosort')


        # Create the xyz array
        xyz = np.c_[mlapdv[:, 0].astype(np.float64) / 1e6, mlapdv[:, 1].astype(np.float64) / 1e6, mlapdv[:, 2].astype(np.float64) / 1e6]

        fit=at.Trajectory.fit(xyz)
        proj=fit.project(xyz)

        # Create a figure with subplots
        fig, axs = plt.subplots(1, 2, figsize=(12, 6),gridspec_kw={'wspace': 0.5})

        # Render the first image
        ax0 = atlas.plot_tilted_slice(xyz, 0, ax=axs[0])
        ax0[0].scatter(proj[:, 1]*1e6, proj[:, 2]*1e6, c='pink', s=5)

        # Render the second image
        ax1 = atlas.plot_tilted_slice(xyz, 1, ax=axs[1])
        ax1[0].scatter(proj[:, 0]*1e6, proj[:, 2]*1e6, c='pink', s=5)

        # Save the plot to a file with a unique name
        filename = f'static/mlapdv_plot_{i}.png'
        fig.savefig(filename, bbox_inches='tight')

        # Return the path to the saved file
        yield eid, filename